# Train and Test the Training Data!
As we discovered in the exploration phase, the kids with an sii score all have taken a questionaire and have had their score taken from that, which leaves the rest of the training data with the kids that don't have sii as they didn't take questionaire or rather their parents did not.<br>
As such we are left with a real training set of 2736 which is not huge, so we'll use cross-validation. <br>
Instinct is to use a nearest neighbours approach, so let's see how that goes.

In [149]:
#set up
import polars as pl
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


pl.Config.set_tbl_rows(50);
pl.Config.set_tbl_width_chars(300);
pl.Config.set_tbl_cols(90);

In [153]:
data = pl.read_csv('data/train.csv')

#Remove the questionaire columns form training
dropCol = [col for col in data.columns if "PCIAT" in col]
dropSeasonCol = [col for col in data.columns if "Season" in col]

data = data.drop(dropCol).drop(dropSeasonCol)
train = data.filter(pl.col('sii').is_not_null()).drop(dropCol).fill_null(-1).fill_null("None")

ColumnNotFoundError: "PCIAT-Season" not found

Resolved plan until failure:

	---> FAILED HERE RESOLVING THIS_NODE <---
DF ["id", "Basic_Demos-Enroll_Season", "Basic_Demos-Age", "Basic_Demos-Sex"]; PROJECT */60 COLUMNS; SELECTION: None

## KNN: Optimal value of K

In [ ]:
#but together a standard pipline
clf = Pipeline(
    steps=[("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=11))]
)

In [7]:
# Split the data into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(train.drop('id','sii'), train.select('sii').to_numpy().flatten(), test_size=0.2, random_state=42)

# Define a range of k values to test
kValues = list(range(3,30))

#Dict to record testing in
kValueDict = {}

# Train and visualize the models with varying k values
for k in kValues:
    # Create KNN model
    #but together a standard pipline
    clf = Pipeline(steps=[("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=k))])

    # Train the model
    clf.fit(Xtrain, ytrain)

    # Make predictions
    ypred = clf.predict(Xtest)

    # Calculate accuracy
    accuracy = accuracy_score(ytest, ypred)

    kValueDict[k] = accuracy



We found that 4 was the optimal number of neighbours. So let's have a base test.

## Create Model & Test

In [10]:
#train the model
clf = Pipeline(steps=[("scaler", StandardScaler()), ("knn", KNeighborsClassifier(n_neighbors=4))])
clf.fit(Xtrain, ytrain)

Pipeline(steps=[('scaler', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=4))])

In [12]:
#get the test data'
testData = pl.read_csv('data/test.csv')

#Remove the questionaire columns form training
dropCol = [col for col in testData.columns if "PCIAT" in col or "Season" in col]
#dropSeasonCol = [col for col in data.columns if "Season" in col]
test = testData.drop(dropCol).fill_null(-1).fill_null("None")

In [14]:
y_pred = clf.predict(test.drop('id'))

In [ ]:
testData.with_columns(pl.Series(y_pred).alias('sii')).select('id', 'sii').write

id,sii
str,i64
"""00008ff9""",0
"""000fd460""",0
"""00105258""",0
"""00115b9f""",0
"""0016bb22""",2
"""001f3379""",0
"""0038ba98""",0
"""0068a485""",0
"""0069fbed""",0


Dispite our o.61 accuracy in our testing we only got 0.282 in the real training which suggests that the survey kids alone do not reflect the testing set well. Not overly suprising given how few observation pionts we have. So let's do a bit of unsupervised clustering and see where all our assigned training data ends up.

### Clustering: All Data

In [134]:
from sklearn.cluster import DBSCAN, AffinityPropagation
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np

In [145]:
#Also want to get rid of a few more things in training
dropCol = [x for x in train.columns if ('FGC-FGC' in x) & ('Zone' not in x)]
trainReduce = data.drop(dropCol)#.select('id', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'SDS-SDS_Total_T', 'PreInt_EduHx-computerinternet_hoursday', 'sii')

In [146]:
#traing data
trainingClust = trainReduce.drop('id', 'sii').fill_null(-1).to_pandas()

In [147]:
clf = Pipeline(steps=[("scaler", StandardScaler()), ("cluster", AffinityPropagation(convergence_iter=25, random_state=42))])
labels = clf.fit_predict(trainingClust)

ValueError: could not convert string to float: 'Fall'

In [136]:
partition = trainReduce.with_columns(pl.Series(labels).alias('Labels')).partition_by('Labels')

In [144]:
trainReduce.with_columns(pl.Series(labels).alias('Labels')).group_by('Labels').agg(pl.col('sii').filter(pl.col('sii').is_null()).len().alias('NullSii'), pl.col('sii').len().alias('TotalCount'), pl.col('sii').filter(pl.col('sii').is_not_null()).mode()).sort('TotalCount')

Labels,NullSii,TotalCount,sii
i64,u32,u32,list[i64]
15,1,1,[]
426,1,1,[]
280,1,1,[]
408,1,1,[]
30,1,1,[]
551,1,1,[]
265,1,1,[]
12,1,1,[]
21,1,1,[]


In [138]:
partition[5]

id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii,Labels
str,i64,i64,i64,i64,i64,i64,i64
"""001f3379""",13,1,50,56,0,1,340
"""02648a1c""",16,1,85,50,0,0,340
"""034f923b""",14,1,51,72,0,0,340
"""09eeec14""",18,1,80,43,null,2,340
"""0afb2535""",14,1,80,40,null,0,340
"""0b7d7aec""",15,1,60,59,0,0,340
"""0ee70b3e""",16,1,60,51,0,1,340
"""18154f3a""",15,1,75,60,0,1,340
"""1da37c60""",17,1,71,76,0,2,340
